# Recurrent Neural Network for movement pattern recognition

To design a model that learns and predicts movement patterns accurately based on the provided dataset, we need to leverage the temporal nature of the data.
This typically involves using Recurrent Neural Networks (RNNs) or their more advanced variants like LSTMs and GRUs, which are well-suited for sequence data.
Here Bidirectional LSTMs and Attention mechanisms to capture the dependencies in the sequences were used. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, Attention, Concatenate, Flatten

## Data preprocessing

In [ ]:
data = pd.read_csv('../../data/dataframes/labels_and_coordinates_preprocessed.csv')

In [ ]:
print(data.columns.tolist())

In [ ]:
# Remove missing values
data = data.dropna()

# Normalize keypoint data
# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()

# Gather all keypoint columns (all columns with a "_x", "_y", "_z", "_v", "_p", "com" or "angle" in their name)
keypoint_columns = [col for col in data.columns if '_x' in col or '_y' in col or '_z' in col or '_v' in col or '_p' in col or 'com' in col or 'angle' in col]
# Apply fit_transform to keypoint column data only
data[keypoint_columns] = scaler.fit_transform(data[keypoint_columns])

# One-hot encode categorical variables
# pd.get_dummies(...) converts categorical variable into dummy/indicator variables (https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)
data = pd.get_dummies(data, columns=['boulder', 'camera', 'participant', 'repetition'])

# Encode labels
if 'label' in data.columns:
    label_encoder = LabelEncoder() # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
    data['label'] = label_encoder.fit_transform(data['label'])

# Parameters for reshaping
timesteps = 2 # Defines how many timesteps each sequence in the dataset will contain. For now it is set to 2, but we should try several options.
total_features = data.drop('label', axis=1).shape[1] # Get nr of total features (= nr of features without the label column)

# A check to ensure that each sequence fed into the model has a consistent shape
if total_features % timesteps != 0:
    raise ValueError(f"Number of total features ({total_features}) is not divisible by defined timesteps ({timesteps}).")
features_per_timestep = total_features // timesteps

## Split and train

In [ ]:
# Reshape data
X = data.drop('label', axis=1).values.reshape(-1, timesteps, features_per_timestep)
y = data['label'].values

# We have to ensure the data types are compatible with TensorFlow
X = X.astype(np.float32)
y = y.astype(np.int32)

# The usual data split into train, test and validation sets...
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# ... and respective TensorFlow train, val and test datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [ ]:
def create_model(timesteps, features_per_timestep, nr_classes):
    """
    This model contains bidirectional LSTMs and self-attention layers
    """
    inputs = Input(shape=(timesteps, features_per_timestep))

    # First Bidirectional LSTM layer with attention
    x1 = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x1 = Dropout(0.3)(x1)
    # Further reading about keras attention layers: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention
    attention_layer_1 = Attention()([x1, x1])  # Self-attention on the first LSTM output

    # Second Bidirectional LSTM layer with attention
    x2 = Bidirectional(LSTM(128, return_sequences=True))(x1)
    x2 = Dropout(0.3)(x2)
    attention_layer_2 = Attention()([x2, x2])  # Self-attention on the second LSTM output

    # Third Bidirectional LSTM layer
    x3 = Bidirectional(LSTM(64, return_sequences=False))(x2)  # No return_sequences to flatten LSTM output
    x3 = Dropout(0.3)(x3)

    # Concatenate attention outputs with the last LSTM layer output
    concatenated = Concatenate()([Flatten()(attention_layer_1), Flatten()(attention_layer_2), x3])

    # Dense layers after concatenation to learn from both the LSTM and attention outputs
    x_final = Dense(128, activation='relu')(concatenated)
    x_final = Dropout(0.3)(x_final)
    x_final = Dense(64, activation='relu')(x_final)
    outputs = Dense(nr_classes, activation='softmax')(x_final)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = create_model(timesteps, features_per_timestep, len(np.unique(y)))
model.summary()


# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1)

## Evaluation

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy}, Test Loss: {test_loss}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Extracting history data
history_dict = history.history
epochs = range(1, len(history_dict['accuracy']) + 1)

# Plotting training and validation accuracy
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, history_dict['accuracy'], 'bo', label='Training accuracy')
plt.plot(epochs, history_dict['val_accuracy'], 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history_dict['loss'], 'bo', label='Training loss')
plt.plot(epochs, history_dict['val_loss'], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

# Predict the labels for the test dataset
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)

# Map numeric labels to their corresponding names
y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# Compute the confusion matrix
cm = confusion_matrix(y_test_labels, y_pred_labels)

# Get unique labels
unique_labels = np.unique(np.concatenate((y_test_labels, y_pred_labels)))

# Set figure size
plt.figure(figsize=(10, 7))

# Generate and display the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(unique_labels), yticklabels=sorted(unique_labels))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)

# Normalize the confusion matrix by row (true labels)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_normalized = np.round(cm_normalized, 2)  # Round to 2 decimal places

# Display the normalized confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=label_encoder.classes_)
fig, ax = plt.subplots(figsize=(14, 14))  # Increase figure size for better readability
disp.plot(xticks_rotation='vertical', ax=ax, cmap='viridis')  # Choose a colormap for better contrast

# Adjust font size
plt.xticks(fontsize=10, ha='center')
plt.yticks(fontsize=10, va='center')

# Update text properties in the matrix
for text in disp.text_.ravel():
    text.set_fontsize(10)  # Set font size for the numbers

plt.tight_layout(pad=3.0)  # Add padding to ensure elements are not overlapping

plt.title('Normalized Confusion Matrix')
plt.show()